# Proyek Machine Learning Pemula
- Nama: Theofilus Arifin
- Email: theofilusarifin@gmail.com
- Id Dicoding: theofilusarifin

## Import Library

In [36]:
import requests
import zipfile
import io

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from matplotlib import pyplot as plt
from keras.preprocessing import image
import tensorflow as tf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

## Data Gathering

In [37]:
# url = "https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip"
# response = requests.get(url)
# with zipfile.ZipFile(io.BytesIO(response.content)) as z:
#     z.extractall()

In [38]:
# Load folder in a directory
directory_path = 'rockpaperscissors'
files = os.listdir(directory_path)

# Load Subfolders
subfolders = [f.path for f in os.scandir(directory_path) if f.is_dir()]

In [39]:
image_data = []
labels = []

# read files
for folder in subfolders:
    label = os.path.basename(folder)
    for filename in os.listdir(folder):
        if filename.endswith('.png'):
            image_path = os.path.join(folder, filename)
            image = cv2.imread(image_path)
            if image is not None:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (150, 150))
                image_data.append(image)
                labels.append(label)

In [40]:
# Convert image data to numpy array
image_data_arr = np.array(image_data)

In [41]:
# Create a label encoder
label_encoder = LabelEncoder()
integer_labels = label_encoder.fit_transform(labels)
one_hot_encoder = OneHotEncoder(sparse=False)
integer_labels = integer_labels.reshape(-1, 1)
labels = one_hot_encoder.fit_transform(integer_labels)

c:\Users\asus\.virtualenvs\learn_dicoding-QPW79iQS\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [42]:
print("Total Image Data:", len(image_data_arr), "Sample")
print("Image Data Shape:", image_data_arr.shape)

Total Image Data: 2188 Sample
Image Data Shape: (2188, 150, 150, 3)


## Preprocessing

In [43]:
# Dataset Split (40% for Validation Set)
X_train, X_test, y_train, y_test = train_test_split(image_data_arr, labels, test_size=0.4, random_state=1)

print("Training Data:", len(X_train), "Sample")
print("Test Data:", len(X_test), "Sample")

Training Data: 1312 Sample
Test Data: 876 Sample


In [44]:
# Train Data Generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='nearest')

train_generator = train_datagen.flow(X_train,  y_train, batch_size=32)

In [45]:
# Validation Data Generator
test_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagen.flow(X_test, y_test, batch_size=32)

## Modeling

In [46]:
# Create Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 17, 17, 128)      

In [47]:
# compile model with adam optimizer and categorical crossentropy
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

In [48]:
# train model
model_history = model.fit(
    train_generator,
    steps_per_epoch=20,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=10,
    verbose=2,
)

Epoch 1/10


## Evaluation

In [ ]:
# accuracy train & validation
plt.plot(model_history.history['accuracy'], label='Training Accuracy')
plt.plot(model_history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.ylabel('Value')
plt.xlabel('Epoch')
plt.legend(loc="lower right")
plt.show()

## Prediction Testing

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():

  path = fn
  img = image.load_img(path, target_size=(150, 150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)

  print(fn)
  if classes[0, 0] != 0:
    print('paper')
  elif classes[0, 1] != 0:
    print('rock')
  else:
    print('scissors')